# Montando um corretor de texto

In [1]:
import pandas as pd#não foi necessário na primeira parte
import nltk#tokenizador
nltk.download('punkt')#necessario para o tokenizador
print('Bibliotecas importadas com sucesso')

Bibliotecas importadas com sucesso


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Construindo nossa base de dados

In [2]:
with open('dados/artigos.txt','r',encoding="utf8") as f:#abrindo sem o pandas(até pq não é um dataframe), importante citar a nossa intenção, no caso 'r' de read, e delimitar o formato de entrada, utf-8(se não dá pau)
    artigos = f.read()#definimos anteriomente a variável como f, agora estamos definindo a variável para exportar dessa função, no caso, artigos
artigos[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

#### Precisamos agora separar as palavras do texto para começar a construção, bem como verificar se temos palavras suficientes

In [3]:
texto_exemplo = 'Olá, tudo bem?'
tokens = texto_exemplo.split()#um método seria utilizar split, mas ele mantem pontuações e caracteres especiais, não sendo eficiente
tokens#importante! esses pedaços de frases são denominados tokens

['Olá,', 'tudo', 'bem?']

In [4]:
len(tokens)#a contagem funciona, mas ainda estamos com caracteres especiais

3

In [5]:
palavras_separadas = nltk.tokenize.word_tokenize(texto_exemplo)#função que gera tokens precisos
palavras_separadas#dividiu melhor, mas será que eu quero ',' e '?'?

['Olá', ',', 'tudo', 'bem', '?']

In [6]:
len(palavras_separadas)#ele está contando os caracteres como palavras

5

In [7]:
'palavra'.isalpha()#Essa função retorna se existem caracteres especiais numa lista de caracteres

True

In [8]:
'palavra?'.isalpha()#caso haja

False

In [9]:
def separa_palavras(lista_de_tokens):#definindo função identificadora
    lista_palavras = []
    for token in lista_de_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

In [10]:
separa_palavras(palavras_separadas)#testando

['Olá', 'tudo', 'bem']

In [11]:
print('iniciando construção da lista de tokens')#demorou pra rodar, criei confirmações para saber onde estava
lista_tokens = nltk.tokenize.word_tokenize(artigos)#criando lista de tokens
print('lista de tokens criada')
lista_palavras = separa_palavras(lista_tokens)#rodando a função
print('lista de palavras criada')
print(f"O número de palavras no texto artigos é:{len(lista_palavras)}")#finalmente!
print('done')# levou um tempo para concluir, queria deixar uma confirmação

iniciando construção da lista de tokens
lista de tokens criada
lista de palavras criada
O número de palavras no texto artigos é:403031
done


In [12]:
def normalizacao(lista):#importante deixar todas as palavras em minusculas para realizar uma comparação
    lista_normalizada = []
    final = len(lista)
    contador = 0
    c1 = final//4#fazendo aparecer % para saber que não travou
    c2 = final//2
    c3 = (final//4)*3
    c4 = final
    print('Iniciando normalização')
    for palavra in lista:
        lista_normalizada.append(palavra.lower())#adicionando a uma segunda lista tudo em minusculo
        contador += 1
        if contador == c1:
            print('25% concluido...')
        if contador == c2:
            print('50% concluido...')
        if contador == c3:
            print('75% concluido...')
        if contador == c4:
            print('100% concluido!') 
    return lista_normalizada#retornando em minusculo(normalizado)

In [13]:
lista_normalizada = normalizacao(lista_palavras)

Iniciando normalização
25% concluido...
50% concluido...
75% concluido...
100% concluido!


In [14]:
print(lista_normalizada[:100])#funcionou

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', 'java', 'para', 'salvar', 'um', 'novo', 'usuário', 'várias', 'validações', 'são', 'feitas', 'como', 'por', 'exemplo', 'ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', 'o', 'cpf', 'só', 'números', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no', 'mínimo', 'anos', 'veja', 'o', 'método', 'que', 'faz', 'essa', 'validação', 'java', 'suponha', 'agora', 'que', 'eu', 'tenha', 'outra', 'classe', 'a', 'classe', 'produto', 'que', 'contém', 'um', 'atributo', 'nome', 'e', 'eu', 'quero', 'fazer', 'a', 'mesma', 'validação', 'que', 'fiz', 'para', 'o', 'nome', 'do', 'usuário', 'ver', 'se', 'só', 'contém', 'letras', 'e', 'aí', 'vou', 'criar', 'outro', 'método', 'para', 'fazer', 'a', 'mesma', 'validação', 'ou', 'criar']


In [15]:
set([0,0,0,1,2,3,3,3,3,3,4,5])#função que remove duplicatas, interessante que não usamos o pandas ainda

{0, 1, 2, 3, 4, 5}

In [16]:
print('Removendo duplicatas')#novamente, medo de travar
lista_limpa = set(lista_normalizada)#atribuindo uma variável
len(lista_limpa)#valor real de palavras unicas no artigo

Removendo duplicatas


18464

## Construindo a correção

#### Primeiro passo é montar um processo eficiente de fatiamento de palavras, para poder inserir caracteres até encontrar o correto

In [17]:
lista = 'lgica'
(lista[:1],lista[1:])#essa função também funciona em strings

('l', 'gica')

In [18]:
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E,D in fatias:   #interessante é que escrevendo dessa forma ele já identifica que são 2 valores em cada tupla e coloca no lugar certo
        for letra in letras:
            nova = str(E)+letra+str(D)
            novas_palavras.append(nova)#remontando a palavra
    return novas_palavras

In [51]:
exemplo = 'lgica'
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):#basicamente mudamos o index, e assim mudamos o local de quebra da palabra
        fatias.append((palavra[:i],palavra[i:]))#importante adicionar +1, já que o ultimo index não é normalmente pego
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

#### Precisamos calcular a probabilidade de uma versão da palavra estar correta

In [58]:
frequencia = nltk.FreqDist(lista_normalizada)#ele calcula quais palavras são mais utilizadas
frequencia.most_common(10)

[('de', 15502),
 ('o', 14056),
 ('que', 12230),
 ('a', 11099),
 ('e', 10501),
 ('para', 7710),
 ('um', 6367),
 ('é', 5899),
 ('uma', 5220),
 ('do', 5124)]

In [21]:
frequencia['lógica']#podemos verificar a frequencia de qualquer palavra

96

In [22]:
exemplo

'lgica'

In [23]:
total_palavras = len(lista_normalizada)#visto anteriormente
total_palavras

403031

In [74]:
def prob(palavra_gerada):
    return frequencia[palavra_gerada]/403031
probabilidade('lógica')#lembrando que total_palavras foi calculado na primeira parte

0.00023819507680550628

In [84]:
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)#trás a lista de possibilidades de fatiamento
    palavra_correta = max(palavras_geradas, key=prob)#escolhe a com maior probabilidade
    return palavra_correta#função max pega o maior valor baseado em uma lista dada, no caso a lista dada na função  gerador de palavras

In [76]:
corretor('lgica')

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica', 'klgica', 'llgica', 'mlgica', 'nlgica', 'olgica', 'plgica', 'qlgica', 'rlgica', 'slgica', 'tlgica', 'ulgica', 'vlgica', 'wlgica', 'xlgica', 'ylgica', 'zlgica', 'àlgica', 'álgica', 'âlgica', 'ãlgica', 'èlgica', 'élgica', 'êlgica', 'ìlgica', 'ílgica', 'îlgica', 'òlgica', 'ólgica', 'ôlgica', 'õlgica', 'ùlgica', 'úlgica', 'ûlgica', 'çlgica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'làgica', 'lágica', 'lâgica', 'lãgica', 'lègica', 'légica', 'lêgica', 'lìgica', 'lígica', 'lîgica', 'lògica', 'lógica', 'lôgica', 'lõgica', 'lùgica', 'lúgica', 'lûgica', 'lçgica', 'lgaica', 'lgbica', 'lgcica', 'lgdica', 'lgeica', 'lgfica', 'lggica', 'lghica', 'lgiica', 'lgjica', 'lgkica', 'lglica',

'lógica'

In [77]:
frequencia['lõgica']

0

## definindo modelo de testagem da versão atual do corretor

In [28]:
def cria_teste(nome_arquivo):
    lista_palavras_teste = []
    f=open(nome_arquivo, 'r',encoding="utf8")
    for linha in f:
        correta,errada = linha.split()
        lista_palavras_teste.append((correta,errada))
    f.close()
    return lista_palavras_teste

lista_teste = cria_teste('dados/palavras.txt')

print(lista_teste[:10])

[('podemos', 'pyodemos'), ('esse', 'esje'), ('já', 'jrá'), ('nosso', 'nossov'), ('são', 'sãêo'), ('dos', 'dosa'), ('muito', 'muifo'), ('imagem', 'iômagem'), ('sua', 'ósua'), ('também', 'tambéùm')]


In [53]:
def avaliador(entrada):
    maximo = len(entrada)
    hits = 0
    palavras_erradas = []
    for certo,errado in entrada:
        if certo == corretor(errado):
            hits += 1
        else:
            palavras_erradas.append((certo,errado))
        taxa = round(hits*100/maximo,2)
        erros = maximo - hits
    print(f' Palavras corrigidas:{hits}')
    print(f' Palavras não corrigidas:{maximo - hits}')
    print(f'Taxa de acerto {taxa}%')
    return(palavras_erradas)
        
avaliador(lista_teste)

 Palavras corrigidas:0
 Palavras não corrigidas:186
Taxa de acerto 0.0%


[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm'),
 ('ele', 'eme'),
 ('fazer', 'èazer'),
 ('temos', 'temfs'),
 ('essa', 'eàssa'),
 ('quando', 'quaôdo'),
 ('vamos', 'vamvos'),
 ('sobre', 'hsobre'),
 ('java', 'sjava'),
 ('das', 'daõs'),
 ('agora', 'agorah'),
 ('está', 'eòtá'),
 ('cada', 'céda'),
 ('mesmo', 'zmesmo'),
 ('nos', 'noâ'),
 ('forma', 'fobma'),
 ('seja', 'sejéa'),
 ('então', 'enêão'),
 ('criar', 'èriar'),
 ('código', 'cóeigo'),
 ('caso', 'casío'),
 ('exemplo', 'áexemplo'),
 ('tem', 'tĩem'),
 ('usuário', 'usuárôio'),
 ('dados', 'dfados'),
 ('python', 'pgthon'),
 ('nossa', 'nossah'),
 ('além', 'alémè'),
 ('assim', 'asõim'),
 ('ter', 'teb'),
 ('até', 'atĩ'),
 ('bem', 'âem'),
 ('design', 'desigen'),
 ('trabalho', 'trabalàho'),
 ('foi', 'foo'),
 ('apenas', 'apenaũ'),
 ('empresa', 'empresà'),
 ('valor', 'valíor'),
 ('será', 'serr')

#### corretor com taxa de correção de 2%

# Adicionando função de remoção de letras

In [30]:
teste = 'lóggica'
def del_letra(fatias):
    novas_palavras = []
    for E, D in fatias:#é esperado vir tuplas
        novas_palavras.append(E + D[1:])#remonta excluindo uma letra
    return novas_palavras

In [31]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):#basicamente mudamos o index, e assim mudamos o local de quebra da palabra
        fatias.append((palavra[:i],palavra[i:]))#importante adicionar +1, já que o ultimo index não é normalmente pego
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += del_letra(fatias)#adicionando essa parte para incluir del de caracteres
    return palavras_geradas
palavras_geradas = gerador_palavras(exemplo)
print(palavras_geradas)

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica', 'klgica', 'llgica', 'mlgica', 'nlgica', 'olgica', 'plgica', 'qlgica', 'rlgica', 'slgica', 'tlgica', 'ulgica', 'vlgica', 'wlgica', 'xlgica', 'ylgica', 'zlgica', 'àlgica', 'álgica', 'âlgica', 'ãlgica', 'èlgica', 'élgica', 'êlgica', 'ìlgica', 'ílgica', 'îlgica', 'òlgica', 'ólgica', 'ôlgica', 'õlgica', 'ùlgica', 'úlgica', 'ûlgica', 'çlgica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'làgica', 'lágica', 'lâgica', 'lãgica', 'lègica', 'légica', 'lêgica', 'lìgica', 'lígica', 'lîgica', 'lògica', 'lógica', 'lôgica', 'lõgica', 'lùgica', 'lúgica', 'lûgica', 'lçgica', 'lgaica', 'lgbica', 'lgcica', 'lgdica', 'lgeica', 'lgfica', 'lggica', 'lghica', 'lgiica', 'lgjica', 'lgkica', 'lglica',

In [32]:
exemplo = ('olharr')
exemplo

'olharr'

In [33]:
corretor(exemplo)

'aolharr'

In [34]:
avaliador(lista_teste)

 Palavras corrigidas:0
 Palavras não corrigidas:186
Taxa de acerto 0.0%


[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm'),
 ('ele', 'eme'),
 ('fazer', 'èazer'),
 ('temos', 'temfs'),
 ('essa', 'eàssa'),
 ('quando', 'quaôdo'),
 ('vamos', 'vamvos'),
 ('sobre', 'hsobre'),
 ('java', 'sjava'),
 ('das', 'daõs'),
 ('agora', 'agorah'),
 ('está', 'eòtá'),
 ('cada', 'céda'),
 ('mesmo', 'zmesmo'),
 ('nos', 'noâ'),
 ('forma', 'fobma'),
 ('seja', 'sejéa'),
 ('então', 'enêão'),
 ('criar', 'èriar'),
 ('código', 'cóeigo'),
 ('caso', 'casío'),
 ('exemplo', 'áexemplo'),
 ('tem', 'tĩem'),
 ('usuário', 'usuárôio'),
 ('dados', 'dfados'),
 ('python', 'pgthon'),
 ('nossa', 'nossah'),
 ('além', 'alémè'),
 ('assim', 'asõim'),
 ('ter', 'teb'),
 ('até', 'atĩ'),
 ('bem', 'âem'),
 ('design', 'desigen'),
 ('trabalho', 'trabalàho'),
 ('foi', 'foo'),
 ('apenas', 'apenaũ'),
 ('empresa', 'empresà'),
 ('valor', 'valíor'),
 ('será', 'serr')

#### Aumentamos bastante a eficiência, somente adicionando possibilidade de deletar letras

## configurando algoritmo de troca de letras

In [35]:
exemplo = 'lígica'

In [36]:
def troca_letra(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E,D in fatias:
        for letra in letras:
               novas_palavras.append(str(E)+letra+str(D[1:]))
    return novas_palavras
        

In [37]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):#basicamente mudamos o index, e assim mudamos o local de quebra da palabra
        fatias.append((palavra[:i],palavra[i:]))#importante adicionar +1, já que o ultimo index não é normalmente pego
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += del_letra(fatias)#adicionando essa parte para incluir del de caracteres
    palavras_geradas += troca_letra(fatias)
    return palavras_geradas

In [38]:
avaliador(lista_teste)

 Palavras corrigidas:0
 Palavras não corrigidas:186
Taxa de acerto 0.0%


[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm'),
 ('ele', 'eme'),
 ('fazer', 'èazer'),
 ('temos', 'temfs'),
 ('essa', 'eàssa'),
 ('quando', 'quaôdo'),
 ('vamos', 'vamvos'),
 ('sobre', 'hsobre'),
 ('java', 'sjava'),
 ('das', 'daõs'),
 ('agora', 'agorah'),
 ('está', 'eòtá'),
 ('cada', 'céda'),
 ('mesmo', 'zmesmo'),
 ('nos', 'noâ'),
 ('forma', 'fobma'),
 ('seja', 'sejéa'),
 ('então', 'enêão'),
 ('criar', 'èriar'),
 ('código', 'cóeigo'),
 ('caso', 'casío'),
 ('exemplo', 'áexemplo'),
 ('tem', 'tĩem'),
 ('usuário', 'usuárôio'),
 ('dados', 'dfados'),
 ('python', 'pgthon'),
 ('nossa', 'nossah'),
 ('além', 'alémè'),
 ('assim', 'asõim'),
 ('ter', 'teb'),
 ('até', 'atĩ'),
 ('bem', 'âem'),
 ('design', 'desigen'),
 ('trabalho', 'trabalàho'),
 ('foi', 'foo'),
 ('apenas', 'apenaũ'),
 ('empresa', 'empresà'),
 ('valor', 'valíor'),
 ('será', 'serr')

#### corretor com taxa de correção de 2%

## Melhoramos bastante, mas ainda podemos corrigir casos onde as letras estão invertidas no texto

In [127]:
def inversor(fatias):
    novas_palavras = []
    for E,D in fatias:
        if len(D) > 1:
            nova = (str(E)+str(D[1])+str(D[0])+str(D[2:]))
            novas_palavras.append(nova)
    return novas_palavras

In [88]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):#basicamente mudamos o index, e assim mudamos o local de quebra da palabra
        fatias.append((palavra[:i],palavra[i:]))#importante adicionar +1, já que o ultimo index não é normalmente pego
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += del_letra(fatias)#adicionando essa parte para incluir del de caracteres
    palavras_geradas += troca_letra(fatias)
    palavras_geradas += inversor(fatias)
    return palavras_geradas

In [89]:
avaliador(lista_teste)

 Palavras corrigidas:142
 Palavras não corrigidas:44
Taxa de acerto 76.34%


[('ele', 'eme'),
 ('nos', 'noâ'),
 ('ter', 'teb'),
 ('bem', 'âem'),
 ('será', 'serr'),
 ('eu', 'eû'),
 ('já', 'jé'),
 ('das', 'dms'),
 ('cada', 'cava'),
 ('nossa', 'nossk'),
 ('eu', 'ìeu'),
 ('ela', 'qelay'),
 ('utilizar', 'dtilizacr'),
 ('projeto', 'bprojõto'),
 ('site', 'ysiteo'),
 ('sem', 'sõêm'),
 ('pelo', 'peàli'),
 ('alura', 'asuraó'),
 ('dia', 'deiìa'),
 ('tudo', 'tuĩdoì'),
 ('ela', 'eúaa'),
 ('utilizar', 'utilizẽaçr'),
 ('projeto', 'prêjetó'),
 ('site', 'sqiqte'),
 ('sem', 'sũexm'),
 ('pelo', 'pçlxo'),
 ('alura', 'uluraa'),
 ('dia', 'dĩaz'),
 ('tudo', 'kzudo'),
 ('empoderamento', 'ewpoderamento'),
 ('cavalo', 'cakvalo'),
 ('canela', 'canelac'),
 ('tênis', 'tênisy'),
 ('ansiosa', 'anciosa'),
 ('ansiosa', 'ancciosa'),
 ('ansiosa', 'ansioa'),
 ('empoderamento', 'empoderamento'),
 ('asterisco', 'asterístico'),
 ('entretido', 'entertido'),
 ('idiota', 'indiota'),
 ('tomara', 'tomare'),
 ('prevalecer', 'provalecer'),
 ('mendigo', 'mindigo'),
 ('perturbar', 'pertubar')]

In [95]:
corretor('armostra')

'amostra'

## Melhorou bastante, mas agora precisamos avaliar o vocabulário

In [100]:
def avaliador(entrada,vocabulario):
    maximo = len(entrada)
    hits = 0
    desconhecida =0
    palavras_erradas = []
    for certo,errado in entrada:
        if certo == corretor(errado):
            hits += 1
        else:
            desconhecida += (certo not in vocabulario)
            palavras_erradas.append((certo,errado))
        taxa = round(hits*100/maximo,2)
        taxa_desconhecida = round(desconhecida*100/maximo,2)#adicionamos isso para descobrir se nosso vocablário esta fraco
        erros = maximo - hits
    print(f' Palavras corrigidas:{hits}')
    print(f' Palavras não corrigidas:{maximo - hits}')
    print(f'Taxa de acerto {taxa}%')
    print(f'Taxa de palavras desconhecidas {taxa_desconhecida}%')
    return(palavras_erradas)
vocabulario = set(lista_normalizada)

In [101]:
avaliador(lista_teste,vocabulario)

 Palavras corrigidas:142
 Palavras não corrigidas:44
Taxa de acerto 76.34%
Taxa de palavras desconhecidas 6.99%


[('ele', 'eme'),
 ('nos', 'noâ'),
 ('ter', 'teb'),
 ('bem', 'âem'),
 ('será', 'serr'),
 ('eu', 'eû'),
 ('já', 'jé'),
 ('das', 'dms'),
 ('cada', 'cava'),
 ('nossa', 'nossk'),
 ('eu', 'ìeu'),
 ('ela', 'qelay'),
 ('utilizar', 'dtilizacr'),
 ('projeto', 'bprojõto'),
 ('site', 'ysiteo'),
 ('sem', 'sõêm'),
 ('pelo', 'peàli'),
 ('alura', 'asuraó'),
 ('dia', 'deiìa'),
 ('tudo', 'tuĩdoì'),
 ('ela', 'eúaa'),
 ('utilizar', 'utilizẽaçr'),
 ('projeto', 'prêjetó'),
 ('site', 'sqiqte'),
 ('sem', 'sũexm'),
 ('pelo', 'pçlxo'),
 ('alura', 'uluraa'),
 ('dia', 'dĩaz'),
 ('tudo', 'kzudo'),
 ('empoderamento', 'ewpoderamento'),
 ('cavalo', 'cakvalo'),
 ('canela', 'canelac'),
 ('tênis', 'tênisy'),
 ('ansiosa', 'anciosa'),
 ('ansiosa', 'ancciosa'),
 ('ansiosa', 'ansioa'),
 ('empoderamento', 'empoderamento'),
 ('asterisco', 'asterístico'),
 ('entretido', 'entertido'),
 ('idiota', 'indiota'),
 ('tomara', 'tomare'),
 ('prevalecer', 'provalecer'),
 ('mendigo', 'mindigo'),
 ('perturbar', 'pertubar')]

## Precisamos agora cobrir os casos onde temos mudanças que não estão lado a lado

In [102]:
def gerador_turbo(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras(palavra)
    return(novas_palavras)

In [104]:
exemplo = 'lóiigica'
result = gerador_turbo(gerador_palavras(exemplo))
exemplo in result

True

In [105]:
len(result)

694980

In [121]:
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)#trás a lista de possibilidades de fatiamento
    palavras_turbo = gerador_turbo(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_turbo)#gerando lista com todas as palavras geradas
    candidatos = [palavra]#iniciando lista em branco, isso permite também que ele mantenha uma palavra correta como correta
    for palavra in todas_palavras:
            if palavra in vocabulario:#comparamos nossa palavra com o vocabulário anteriormente definido, não gosto de deixar variável global aqui, mas paciência
                candidatos.append(palavra)
    palavra_correta = max(candidatos, key=prob)#escolhe a com maior probabilidade
#    print(f'{len(candidatos)} candidatos a palavra correta')
#    print(candidatos)
    return palavra_correta#função max pega o maior valor baseado em uma lista dada, no caso a lista dada na função  gerador de palavras

In [122]:
novo_corretor(exemplo)

'lógica'

In [129]:
print('testando novo corretor')
def avaliador(entrada,vocabulario):
    maximo = len(entrada)
    hits = 0
    desconhecida =0
    palavras_erradas = []
    for certo,errado in entrada:
        desconhecida += (certo not in vocabulario)
        if certo == novo_corretor(errado):
            hits += 1
        else:
            palavras_erradas.append((certo,errado))
        taxa = round(hits*100/maximo,2)
        taxa_desconhecida = round(desconhecida*100/maximo,2)#adicionamos isso para descobrir se nosso vocablário esta fraco
        erros = maximo - hits
    print(f' Palavras corrigidas:{hits}')
    print(f' Palavras não corrigidas:{maximo - hits}')
    print(f'Taxa de acerto {taxa}%')
    print(f'Taxa de palavras desconhecidas {taxa_desconhecida}%')
    return(palavras_erradas)

testando novo corretor


In [130]:
print('testando novo corretor')
avaliador(lista_teste,vocabulario)
print('done')

testando novo corretor
 Palavras corrigidas:103
 Palavras não corrigidas:83
Taxa de acerto 55.38%
Taxa de palavras desconhecidas 6.99%
done


#### O corretor piorou, interessante!, tendo em vista isso, iremos retornar a versão base, que tinha maior eficiência

In [134]:
print(avaliador(lista_teste,vocabulario))

 Palavras corrigidas:103
 Palavras não corrigidas:83
Taxa de acerto 55.38%
Taxa de palavras desconhecidas 6.99%
[('esse', 'esje'), ('são', 'sãêo'), ('dos', 'dosa'), ('ele', 'eme'), ('essa', 'eàssa'), ('das', 'daõs'), ('cada', 'céda'), ('nos', 'noâ'), ('então', 'enêão'), ('tem', 'tĩem'), ('nossa', 'nossah'), ('ter', 'teb'), ('até', 'atĩ'), ('bem', 'âem'), ('foi', 'foo'), ('será', 'serr'), ('entre', 'entke'), ('vai', 'van'), ('seus', 'çeus'), ('eu', 'eû'), ('tempo', 'temeo'), ('sempre', 'semre'), ('ela', 'elaá'), ('só', 'síó'), ('site', 'siàe'), ('sem', 'seém'), ('pelo', 'peln'), ('alura', 'aléra'), ('dia', 'tdia'), ('tudo', 'tuúo'), ('já', 'jé'), ('são', 'sãô'), ('dos', 'odos'), ('sua', 'siua'), ('ele', 'elpe'), ('temos', 'teos'), ('essa', 'eũsa'), ('vamos', 'vjmos'), ('das', 'dms'), ('cada', 'cava'), ('nos', 'ános'), ('caso', 'èaso'), ('tem', 'túem'), ('dados', 'daáos'), ('nossa', 'nossk'), ('ter', 'tãer'), ('até', 'vté'), ('bem', 'búm'), ('será', 'sçerá'), ('entre', 'entró'), ('vai', 

In [135]:
print('testando novo corretor')
def avaliador(entrada,vocabulario):
    maximo = len(entrada)
    hits = 0
    desconhecida =0
    palavras_erradas = []
    for certo,errado in entrada:
        desconhecida += (certo not in vocabulario)
        if certo == corretor(errado):
            hits += 1
        else:
            palavras_erradas.append((certo,errado))
        taxa = round(hits*100/maximo,2)
        taxa_desconhecida = round(desconhecida*100/maximo,2)#adicionamos isso para descobrir se nosso vocablário esta fraco
        erros = maximo - hits
    print(f' Palavras corrigidas:{hits}')
    print(f' Palavras não corrigidas:{maximo - hits}')
    print(f'Taxa de acerto {taxa}%')
    print(f'Taxa de palavras desconhecidas {taxa_desconhecida}%')
    return(palavras_erradas)

testando novo corretor


In [137]:
palavra = "lgiica"
print(novo_corretor(palavra))
print(corretor(palavra))

lógica
algiica
